<a href="https://colab.research.google.com/github/areebarshad/multi-asset-portfolio-optimizer/blob/main/notebooks/DataAcquisition_and_Preprocessing/DataAcquisition_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#define multi-asset tickers
tickers = ['VEA', 'VWO', 'EWJ', 'TLT', 'LQD', 'GLD', 'VNQ', 'DBC']

#download the closing prices and save to csv
data = yf.download(tickers, start = "2015-01-01", end = "2025-12-31")['Close']
data.to_csv('/content/multiasset_closing_prices.csv')

#output a fitting message
print("Data collected and shaped: ", data.shape)
print(data.head())

#check for missing values
print(data.isnull().sum())

#forward fill then backfill if NaNs remain
data = data.fillna(method = 'ffill').fillna(method = 'bfill')

#confirm no missing data remains
print(data.isnull().sum().sum())

#calculate daily returns and save to csv
daily_returns = np.log(data / data.shift(1)).dropna()
daily_returns.to_csv('/content/multiasset_daily_returns.csv')

#calculate monthly returns
monthly_prices = data.resample('M').last()
monthly_returns = np.log(monthly_prices / monthly_prices.shift(1)).dropna()

#plot the closing prices
plt.figure(figsize = (12, 6))
for ticker in tickers:
  plt.plot(data[ticker], label = ticker)
plt.legend()
plt.title("Closing Prices")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()

#plot daily return distributions
daily_returns.plot(kind = 'hist', bins = 100, alpha = 0.7, figsize = (12, 6))
plt.title("Distribution of Daily Returns")
plt.xlabel("Daily Log Return")
plt.show()

#plot the correlation heatmap
plt.figure(figsize = (12, 6))
sns.heatmap(daily_returns.corr(), annot = True, cmap = 'coolwarm')
plt.title("Correlation Heatmap of Daily Returns")
plt.show()

#calculate the mean and standard deviation of daily returns
stats = pd.DataFrame({'Mean of Daily Returns': daily_returns.mean(),
                      'Standard Deviation of Daily Returns': daily_returns.std()})

#annualize mean and std (assumption: 252 trading days)
stats['Annualized Return'] = stats['Mean of Daily Returns'] * 252
stats['Annualized Volatility'] = stats["Standard Deviation of Daily Returns"] * np.sqrt(252)

#display stats and save to csv
print(stats)
stats.to_csv('/content/multiasset_stats.csv')